## Part 2

1. Write a function that estimates the emission parameters `e(x|y)` from the training set using MLE (maximum likelihood estimation).

In [1]:
import utils
import emission

In [2]:
dataset_folder = "data/EN/"
train_data = dataset_folder + "train"
lines = utils.read_file_to_lines(train_data)

In [3]:
emission_data = emission.generate_emission_table(lines)

Skipped 1 lines: [''] 



In [4]:
# example: e(word|tag) aka tag -> word

word = "Municipal"
tag = "B-NP"

print("MLE: e(", word, "|", tag, ") =", emission.get_mle(word, tag, emission_data))

MLE: e( Municipal | B-NP ) = 2.1139414438220062e-05


2. Use label smoothing with `k=3` to account for rare and unknown words.

In [5]:
# tests

hashmap = emission_data["x_hashmap"]
og_vocab_size = len(hashmap.keys())

# k = 0
hashmap_smoothed = utils.add_unk(hashmap, k=0)
assert len(hashmap.keys()) == len(hashmap_smoothed.keys())

# k = 1 > k = 2
hashmap_k1 = utils.add_unk(hashmap, k=1)
hashmap_k2 = utils.add_unk(hashmap, k=2)
assert len(hashmap_k1.keys()) > len(hashmap_k2.keys())

In [6]:
smoothed_hashmap = utils.add_unk(hashmap, k=3)
smooth_vocab_size = len(smoothed_hashmap.keys())
print("Reduced vocab from", og_vocab_size, "to", smooth_vocab_size)

Reduced vocab from 18213 to 4829


In [7]:
emission_data["x_hashmap"] = smoothed_hashmap
print("MLE: e(", word, "|", tag, ") =", emission.get_mle(word, tag, emission_data))

MLE: e( Municipal | B-NP ) = 0.10364654899059296


3. Implement a simple sentiment analysis system

In [8]:
eval_data = dataset_folder + "dev.in"
lines = utils.read_file_to_lines(eval_data)
outfile = dataset_folder + "dev.p2.out"

pos_tags = list(emission_data["y_tags"].keys())

preds = []

for word in lines:
    if word is not "":
        mle_list = [emission.get_mle(word, tag, emission_data) for tag in pos_tags]
        argmax_mle = pos_tags[mle_list.index(max(mle_list))]
        preds.append(argmax_mle)
    else:
        preds.append("")
    
output = zip(lines, preds)

with open(outfile, "w") as f:
    for line in output:
        f.write(line[0] + " " + line[1] +"\n")

In [9]:
!python3 evalResult.py ./data/EN/dev.out ./data/EN/dev.p2.out


#Entity in gold data: 13179
#Entity in prediction: 19580

#Correct Entity : 9055
Entity  precision: 0.4625
Entity  recall: 0.6871
Entity  F: 0.5528

#Correct Sentiment : 7431
Sentiment  precision: 0.3795
Sentiment  recall: 0.5639
Sentiment  F: 0.4537
